In [ ]:
import pandas as pd
import numpy as np 
import networkx as nx
import time
import copy
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.matutils import cossim, jaccard
from scipy import stats

In [ ]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, r
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()

In [ ]:
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Getting data and Stopword Creation

In [ ]:

#                                                '''Edgelist import'''
citation_edges = pd.read_csv('C:/Users/Pratyush Yadav/Desktop/IITM/Research/IPM Exp/AMiner_IS/JOI_Revise/cite_edges.csv')
# citation_edges = pd.read_csv('/Users/pratyushyadav/Desktop/Citation Final/AMiner/Physics/Nodes1/Data/cite_edges.csv')
# citation_edges = pd.read_csv('/Users/pratyushyadav/Desktop/Citation Final/AMiner/Mops/Nodes1/Data/cite_edges.csv')

                        
#                    '''filtered attributes with n_ref and n_cite, used for PCA''' 
pca_factors2 = pd.read_pickle('C:/Users/Pratyush Yadav/Desktop/IITM/Research/IPM Exp/AMiner_IS/new_pca_results.pkl')
# cluster = pd.read_pickle('/Users/pratyushyadav/Desktop/Citation Final/AMiner/Physics/Nodes1/Data/Filtered_Data.pkl')
# cluster = pd.read_pickle('/Users/pratyushyadav/Desktop/Citation Final/AMiner/Mops/Nodes1/Data/Filtered_Data.pkl')


#                                '''input data (original subsetted data)'''
citation_data = pd.read_csv('C:/Users/Pratyush Yadav/Downloads/Papers.csv')
# citation_data = pd.read_pickle('/Users/pratyushyadav/Desktop/Citation Final/AMiner/Physics/Nodes1/Data/AMiner_physics.pkl')
# citation_data = pd.read_pickle('/Users/pratyushyadav/Desktop/Citation Final/AMiner/Mops/Nodes1/Data/AMiner_mops.pkl')


#                                    '''Creation of global graph'''
# G1=nx.from_pandas_edgelist(citation_edges,'Source','Target','weight',create_using=nx.DiGraph())
# G2=nx.from_pandas_edgelist(citation_edges,'Source','Target',create_using=nx.DiGraph())
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown","study","algorithm","paper","method","view","approach","research","data","study","interest","propose"]
stop_words = stop_words.union(new_words)

In [ ]:
import ast
citation_data['references']=citation_data['references'].apply(lambda x: ast.literal_eval(x))

In [ ]:
G1=nx.from_pandas_edgelist(citation_edges,'Source','Target','weight_best',create_using=nx.DiGraph())
G2 = nx.from_pandas_edgelist(citation_edges,'Source','Target','weight_best',create_using=nx.Graph())

In [ ]:
node_5 = cluster[(cluster['n_ref']>=3)]['Paper_Id'].tolist()

### Community Detection and Target Papers

In [ ]:
from community import community_louvain
partition = community_louvain.best_partition(G2)

In [ ]:
key=[]
values=[]
for i in partition.items():
    key.append(i[0])
    values.append(i[1])

In [ ]:
import networkx as nx
community_louvain.modularity(partition,G2)

In [ ]:
Modularity['modularity class'].nunique()

In [ ]:
Modularity = pd.DataFrame({
    'Id':key,
    'modularity class':values
})
Modularity.to_csv('C:/Users/Pratyush Yadav/Desktop/IITM/Research/IPM Exp/AMiner_IS/JOI_Revise/Modularity_new.csv',index=False)

### Function Definitions

In [ ]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    lst4 = len(lst3)
    return lst4

### Creating Ego and Popularity Network

In [ ]:
def create_ego_new(node,G,r):
#     if (G.order()>30):
    G_ego = nx.ego_graph(G,node,undirected=True,radius=math.ceil(r))
    if (G_ego.order()<30):
        G_ego=nx.ego_graph(G,node,undirected=True,radius=r,distance='weight_best')
# #                     if (G_ego.order()<30):
# #                         G_ego = nx.ego_graph(G,node,undirected=True,radius=math.ceil(2*r),distance='weight')
#     else:
#         G_ego=G
    if (G_ego.order()<5):
        G_ego=G
    return G_ego

def popularity_graph_test(G,citation_data,Node,test):
    edge_list1 =[]
    for line in nx.generate_edgelist(G, data=['weight_best']):
        edge_list1.append(line)
    source=[]
    target=[]
    weight=[]
    for item in edge_list1:
        if (item.split(" ")[0]==Node):
            if item.split(" ")[1] not in test:
                source.append(item.split(" ")[0])
                target.append(item.split(" ")[1])
                weight.append(float(item.split(" ")[2]))
        else:
            source.append(item.split(" ")[0])
            target.append(item.split(" ")[1])
            weight.append(float(item.split(" ")[2]))
    cite_edges2 = pd.DataFrame({
        'Source':source,
        'Target':target,
        'weight':weight
    })
    cite_edges2 =cite_edges2.sort_values(['weight'],ascending=True)
    #cite_edges2['weight']= (cite_edges2['weight']-cite_edges2['weight'].min())/(cite_edges2['weight'].max()-cite_edges2['weight'].min())
    '''Source1=[]
    Target1=[]
    Weight1=[]
    for i in range(len(cite_edges2)):
        if (cite_edges2.weight[i]<0):
            Source1.append(cite_edges2.Target[i])
            Target1.append(cite_edges2.Source[i])
            Weight1.append(0-cite_edges2.weight[i])
        else:
            Source1.append(cite_edges2.Source[i])
            Target1.append(cite_edges2.Target[i])
            Weight1.append(cite_edges2.weight[i])
    pop_edges= pd.DataFrame({
        'Source':Source1,
        'Target':Target1,
        'Weight':Weight1
    })'''
    Pop_G=nx.from_pandas_edgelist(cite_edges2,'Source','Target','weight',create_using=nx.DiGraph())
    return Pop_G

### Content Similarity Calculation

In [ ]:
citation_data['paper_id'] = citation_data['paper_id'].apply(lambda x: str(x))
citation_data['Abstract'].fillna('',inplace=True)

In [ ]:
def content_similarity(Pop_G,citation_data,Node):    
    node3=[]
    for node in Pop_G.nodes():
        node3.append(str(node))
    citation_data1=citation_data[citation_data['paper_id'].isin(node3)]
    citation_data1.index = range(len(citation_data1))
    #for content similarity calculation
    LSI_data=[]
    paper_id2=[]
    for i in range(len(citation_data1)):
        if(citation_data1.Abstract[i]!=''):
            LSI_data.append(citation_data1.Abstract[i])
        else:
            LSI_data.append(citation_data1.Title[i])
        paper_id2.append(citation_data1.paper_id[i])
    Data_for_LSI = pd.DataFrame({
        'Paper_ID':paper_id2,
        'LSI_Data':LSI_data
    })
    processed_docs = Data_for_LSI['LSI_Data'].map(preprocess)
    dictionary = gensim.corpora.Dictionary(processed_docs)
    count = 0
    for k, v in dictionary.iteritems():
        #print(k, v)
        count += 1
        if count > 10:
            break
#     print(len(dictionary))
    dictionary2 = copy.deepcopy(dictionary)
    dictionary2.filter_extremes(no_below=15, no_above=0.5,keep_n=100000)
#     print(len(dictionary2))
    bow_corpus = [dictionary2.doc2bow(doc) for doc in processed_docs]
    #bow_corpus[4310]
    tfidf = gensim.models.TfidfModel(bow_corpus)  
    #doc_bow = [(0, 1), (1, 1)]
    #print(tfidf[doc_bow]) 
    corpus_tfidf = tfidf[bow_corpus]
#     print(corpus_tfidf)
    if (len(dictionary2)==0):
        bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
        tfidf = gensim.models.TfidfModel(bow_corpus)  
        corpus_tfidf = tfidf[bow_corpus]
        lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)  # initialize an LSI transformation
        corpus_lsi = lsi[corpus_tfidf]
        doc = citation_data[citation_data['paper_id']==str(Node)]['Title'].tolist()[0]
        vec_bow = dictionary.doc2bow(doc.lower().split())
        vec_lsi = lsi[vec_bow]
        index = gensim.similarities.MatrixSimilarity(lsi[corpus_tfidf])
        if (len(vec_lsi)==0):
            doc = citation_data[citation_data['paper_id']==str(Node)]['Abstract'].tolist()[0]
            vec_bow = dictionary.doc2bow(doc.lower().split())
            vec_lsi = lsi[vec_bow]
            index = gensim.similarities.MatrixSimilarity(lsi[corpus_tfidf])
        #vec_lda = lda[vec_bow]
        #print(vec_lsi)
        if (len(vec_lsi)==0):
            bow_corpus = [dictionary1.doc2bow(doc) for doc in processed_docs]
            tfidf = gensim.models.TfidfModel(bow_corpus)  
            corpus_tfidf = tfidf[bow_corpus]
            doc = citation_data[citation_data['paper_id']==str(Node)]['Title'].tolist()[0]
            vec_bow = dictionary1.doc2bow(doc.lower().split())
            vec_lsi = lsi1[vec_bow]
            index = gensim.similarities.MatrixSimilarity(lsi1[corpus_tfidf])
    else:
        lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary2, num_topics=10)  # initialize an LSI transformation
        corpus_lsi = lsi[corpus_tfidf]
        doc = citation_data[citation_data['paper_id']==str(Node)]['Title'].tolist()[0]
        vec_bow = dictionary2.doc2bow(doc.lower().split())
        vec_lsi = lsi[vec_bow] # convert the query to LSI space
        index = gensim.similarities.MatrixSimilarity(lsi[corpus_tfidf])
        if (len(vec_lsi)==0):
            bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
            tfidf = gensim.models.TfidfModel(bow_corpus)  
            corpus_tfidf = tfidf[bow_corpus]
            lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)  # initialize an LSI transformation
            corpus_lsi = lsi[corpus_tfidf]
            doc = citation_data[citation_data['paper_id']==str(Node)]['Title'].tolist()[0]
            vec_bow = dictionary.doc2bow(doc.lower().split())
            vec_lsi = lsi[vec_bow]
            index = gensim.similarities.MatrixSimilarity(lsi[corpus_tfidf])
            if (len(vec_lsi)==0):
                doc = citation_data[citation_data['paper_id']==str(Node)]['Abstract'].tolist()[0]
                vec_bow = dictionary.doc2bow(doc.lower().split())
                vec_lsi = lsi[vec_bow]
                index = gensim.similarities.MatrixSimilarity(lsi[corpus_tfidf])
            #vec_lda = lda[vec_bow]
            #print(vec_lsi)
            if (len(vec_lsi)==0):
                bow_corpus = [dictionary1.doc2bow(doc) for doc in processed_docs]
                tfidf = gensim.models.TfidfModel(bow_corpus)  
                corpus_tfidf = tfidf[bow_corpus]
                doc = citation_data[citation_data['paper_id']==str(Node)]['Title'].tolist()[0]
                vec_bow = dictionary1.doc2bow(doc.lower().split())
                vec_lsi = lsi1[vec_bow]
                index = gensim.similarities.MatrixSimilarity(lsi1[corpus_tfidf])
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    pap_id1=[]
    sim_score=[]
    for i in range(len(sims)):
        pap_id1.append(citation_data1[citation_data1.index==sims[i][0]]['paper_id'].tolist()[0])
        sim_score.append(sims[i][1])
    Content_sim = pd.DataFrame({
        'Paper_ID':pap_id1,
        'Similarity':sim_score
    })
#     Content_sim= Content_sim[Content_sim['Paper_ID']!=str(Node)]
#     Content_sim.index = range(len(Content_sim))
    #Content_sim['Similarity'] = (Content_sim['Similarity']-Content_sim['Similarity'].min())/(Content_sim['Similarity'].max()-Content_sim['Similarity'].min())
    return Content_sim
def preprocess(doc):#for i in range(len(citation_data.abstract)):
    result=[]
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', doc)

    #Convert to lowercase
    text = text.lower()

    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    ##Convert to list from string
    text = text.split()

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    #data_lemmatized = lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    #return data_lemmatized


    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    for word in text:
        if word not in stop_words:
            lem = WordNetLemmatizer()
            text = lem.lemmatize(word) 
            result.append(text)
    return result 

In [ ]:
textstem =importr('textstem')
base = importr('base')

In [ ]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ

#     elif nltk_tag.startswith('V'):
#         return wordnet.VERB

    if nltk_tag.startswith('N'):
        return wordnet.NOUN

#     elif nltk_tag.startswith('R'):
#         return wordnet.ADV

    else:          
        return None
# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}

#     return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_sentence(sentence):
    words = np.array([i.lower() for i in sentence.split() if i not in stop_words])
    lemmatized_sentence = textstem.lemmatize_words(words).tolist()   
    nltk_tagged = nltk.pos_tag(lemmatized_sentence)
    wordnet_tagged = wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     nltk_tagged = nltk.pos_tag(tokens)
#     tokens = [i.lower() for i in nltk.word_tokenize(sentence)]
#     nltk_tagged = nltk.pos_tag(tokens)
#     wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     lemmatized_sentence = []

#     for word, tag in wordnet_tagged:
#         if word not in stop_words:
#             if tag is None:
#                 continue
#             else:
#                 if word.endswith('ing'):
#                     lemmatized_sentence.append(lemmatizer.lemmatize(word, wordnet.VERB))
#                 else:
#                     lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_final=[]
    for word, tag in wordnet_tagged:
        if tag is not None:
            lemmatized_final.append(word)

    return lemmatized_final

def clean_text(abstract):
    abstract = abstract.replace(". ", " ").replace(", ", " ").replace("! ", " ")\
                       .replace("? ", " ").replace(": ", " ").replace("; ", " ")\
                       .replace("( ", " ").replace(") ", " ").replace("| ", " ").replace("/ ", " ").replace("'","")
    if "." in abstract or "," in abstract or "!" in abstract or "?" in abstract or ":" in abstract or ";" in abstract or "(" in abstract or ")" in abstract or "|" in abstract or "/" in abstract:
        abstract = abstract.replace(".", " ").replace(",", " ").replace("!", " ")\
                           .replace("?", " ").replace(":", " ").replace(";", " ")\
                           .replace("(", " ").replace(")", " ").replace("|", " ").replace("/", " ").replace("'","")
    abstract = abstract.replace("  ", " ")
    
#     for word in list(set(stopwords.words("english"))):
#         abstract = abstract.replace(" " + word + " ", " ")


    return lemmatize_sentence(abstract)

### Offline Computations

In [ ]:
nodes = [int(i) for i in node_5]
mod_classes=Modularity[Modularity['Id'].isin(nodes)]['modularity class'].unique().tolist()
len(mod_classes)

In [ ]:
n_node=[]
radius=[]
for i in range(len(mod_classes)):
    node_list1 = Modularity[Modularity['modularity class']==mod_classes[i]]['Id'].tolist()
    G_sub = G1.subgraph(node_list1)
    sum1=0
    n_path=0
    for j in nx.shortest_path_length(G_sub):
        for a,b in j[1].items():
            sum1=sum1+b
            if (b!=0):
                n_path=n_path+1
    r = sum1/n_path  
    radius.append(r/2)
    n_node.append(len(node_list1))
    print(i)

In [ ]:
Community_info=pd.DataFrame({
    'modularity class':mod_classes,
    'Radius':radius,
    'Nodes':n_node
})

In [ ]:
mod_class=[]
node_info=[]
n_node=[]
radius=[]
for i in range(len(node_5)):
    Node = int(node_5[i])
    class1=Modularity[Modularity['Id']==Node]['modularity class'].tolist()[0]
    node_info.append(Node)
    mod_class.append(class1)
    radius.append(Community_info[Community_info['modularity class']==class1]['Radius'].tolist()[0])
    n_node.append(Community_info[Community_info['modularity class']==class1]['Nodes'].tolist()[0])

In [ ]:
Modularity_1=pd.DataFrame({
    'Node':node_info,
    'Classes':mod_class,
    'Nodes':n_node,
    'Radius':radius
})
# Modularity_1#.to_pickle('/Users/pratyushyadav/Desktop/Citation Final/AMiner/IS&CV/Nodes1/Data/Modularity_1.pkl')

In [ ]:
# #for evaluation purposes (general corpus)
LSI_data1=[]
paper_id3=[]
for i in range(len(citation_data)):
    if(citation_data.Abstract[i]!=''):
        LSI_data1.append(citation_data.Abstract[i])
    else:
        LSI_data1.append(citation_data.Title[i])
    paper_id3.append(citation_data.paper_id[i])
#for evaluation
Data_for_LSI1 = pd.DataFrame({
    'Paper_ID':paper_id3,
    'LSI_Data':LSI_data1
})
processed_docs1 = Data_for_LSI1['LSI_Data'].map(preprocess)
dictionary1 = gensim.corpora.Dictionary(processed_docs1)
count1 = 0
for k, v in dictionary1.iteritems():
    print(k, v)
    count1 += 1
    if count1 > 10:
        break


In [ ]:
bow_corpus1 = [dictionary1.doc2bow(doc) for doc in processed_docs1]
#bow_corpus[4310]
tfidf1 = gensim.models.TfidfModel(bow_corpus1)  
doc_bow = [(0, 1), (1, 1)]
print(tfidf1[doc_bow]) 
corpus_tfidf1 = tfidf1[bow_corpus1]
lsi1 = gensim.models.LsiModel(corpus_tfidf1, id2word=dictionary1, num_topics=10)  # initialize an LSI transformation
corpus_lsi1 = lsi1[corpus_tfidf1]

### Main Function

In [ ]:
import statistics
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)

In [ ]:
import numpy as np

def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
cluster['Recency1'] = normalize(cluster['Recency'])

In [ ]:
pca_factors2['Quality1'] = normalize(pca_factors2['score_best'])

In [ ]:
alpha = 0.1
beta = 0.1
gamma = 0.8

In [ ]:
alpha

In [ ]:
time_exec1=[]
prec1=[]
recall1=[]
ave_prec3=[]
rank3=[]
ave_prec4=[]
rank4=[]
pap_ids=[]
for i in range(2000):
    Node = int(node_5[i]) #int values not str
    print(Node, i)
    #mod_class = Modularity[Modularity['Id']==Node]['modularity class'].tolist()[0]
    #node_list1 = Modularity[Modularity['modularity class']==mod_class]['Id'].tolist()
    list1 = citation_data[citation_data['paper_id']==str(Node)]['references'].tolist()[0]
    mod_class = Modularity_1[Modularity_1['Node']==Node]['Classes'].tolist()[0]
    if(type(mod_class)==int):
        node_list1 = Modularity[Modularity['modularity class']==mod_class]['Id'].tolist()
    else:
        node_list1 = Modularity[Modularity['modularity class'].isin(mod_class)]['Id'].tolist()
    r = Modularity_1[Modularity_1['Node']==Node]['Radius'].tolist()[0]
    G_sub = G1.subgraph(node_list1)
    prec=[]
    recall=[]
    time_exec=[]
    ave_prec1=[]
    ave_prec2=[]
    rank1=[]
    rank2=[]
    for train_index, test_index in kf.split(list1):
        test=[]
        for i in test_index:
            test.append(list1[i])
#         tests.append(test)
        start_time = time.time()
        G_ego = create_ego_new(Node,G_sub,r)
        Pop_G = popularity_graph_test(G_ego,citation_data,str(Node),test)
        if (Pop_G.order()!=G_ego.order()):
            for i in range(len(test)):
                if int(test[i]) in list(G_ego.nodes()):
                    Pop_G.add_node(test[i])
        Content_sim = content_similarity(Pop_G,citation_data,Node)
        recency=[]
        popul=[]
        for i in range(len(Content_sim)):
            recency.append(cluster[cluster['Paper_Id']==int(Content_sim['Paper_ID'][i])]['Recency1'].tolist()[0])
            popul.append(pca_factors2[pca_factors2['paper_id']==Content_sim['Paper_ID'][i]]['Quality1'].tolist()[0])
        Content_sim['Recency1'] = recency
        Content_sim['Quality1']= popul
        Content_sim['Similarity1']=normalize(Content_sim['Similarity'])
        Content_sim.fillna(0,inplace=True)
#         Content_sim['Final_Score1'] = (Content_sim['Similarity1']+Content_sim['Recency1']+Content_sim['Quality1'])/3
        Content_sim['Final_Score1'] = alpha*Content_sim['Similarity1']+beta*Content_sim['Recency1']+gamma*Content_sim['Quality1']
        personalization = Content_sim.set_index('Paper_ID')['Final_Score1'].to_dict()
#         if (Content_sim['Similarity'].unique().tolist()[0]!=0.0):
#             personalization = Content_sim.set_index('Paper_ID')['Similarity'].to_dict()
#         else:
#             personalization=None
        try:
            calculated_page_rank = nx.pagerank(Pop_G, alpha = 0.35,weight='weight',personalization=personalization
                                                  ,dangling=None)
        except:
            calculated_page_rank = nx.pagerank(Pop_G, alpha = 0.35,weight='weight',personalization=personalization,dangling=None)
    # # most important sentences in descending order of importance
        sentences = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)
        if (str(Node) in sentences):
            sentences.remove(str(Node))
        time_exec.append((time.time()-start_time))
  #      print(time_exec)
        paper_1=sentences[0:3]
        paper_2=sentences[0:5]
        paper_3=sentences[0:10]
        paper_4=sentences[0:15]
        paper_5=sentences[0:20]
        paper_6=sentences[0:100]
        pap_ids.append(paper_6)
        len1 = intersection(paper_1,test)
        len2 = intersection(paper_2,test)
        len3 = intersection(paper_3,test)
        len4 = intersection(paper_4,test)
        len5 = intersection(paper_5,test)
        exact_hit1=[(len1/len(test)),(len2/len(test)),(len3/len(test)),(len4/len(test)),(len5/len(test))]
        exact_hit2=[len1/3,len2/5,len3/10,len4/15,len5/20]
        len6 = intersection(paper_1,list1)
        len7 = intersection(paper_2,list1)
        len8 = intersection(paper_3,list1)
        len9 = intersection(paper_4,list1)
        len10 = intersection(paper_5,list1)
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        for i in range(len(paper_1)):
            if paper_1[i] in test:
                m=m+1
                list_pr_1.append(m/(i+1))
                if (n<2):
                    rank_cv.append(1/(i+1))
                    n=n+1
        if (m!=0):
            ave_prec1.append(sum(list_pr_1)/m)
        else:
            ave_prec1.append(0)
        rank1.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        j=0
        for i in range(len(paper_1)):
            if paper_1[i] in list1:
                if paper_1[i] in test:
                    m=m+1
                    j=j+1
                    list_pr_1.append(m/j)
                    if (n<2):
                        rank_cv.append(1/j)
                        n=n+1
            else:
                j=j+1
        if (m!=0):
            ave_prec2.append(sum(list_pr_1)/m)
        else:
            ave_prec2.append(0)
        rank2.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        for i in range(len(paper_2)):
            if paper_2[i] in test:
                m=m+1
                list_pr_1.append(m/(i+1))
                if (n<2):
                    rank_cv.append(1/(i+1))
                    n=n+1
        if (m!=0):
            ave_prec1.append(sum(list_pr_1)/m)
        else:
            ave_prec1.append(0)
        rank1.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        j=0
        for i in range(len(paper_2)):
            if paper_2[i] in list1:
                if paper_2[i] in test:
                    m=m+1
                    j=j+1
                    list_pr_1.append(m/j)
                    if (n<2):
                        rank_cv.append(1/j)
                        n=n+1
            else:
                j=j+1
        if (m!=0):
            ave_prec2.append(sum(list_pr_1)/m)
        else:
            ave_prec2.append(0)
        rank2.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        for i in range(len(paper_3)):
            if paper_3[i] in test:
                m=m+1
                list_pr_1.append(m/(i+1))
                if (n<2):
                    rank_cv.append(1/(i+1))
                    n=n+1
        if (m!=0):
            ave_prec1.append(sum(list_pr_1)/m)
        else:
            ave_prec1.append(0)
        rank1.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        j=0
        for i in range(len(paper_3)):
            if paper_3[i] in list1:
                if paper_3[i] in test:
                    m=m+1
                    j=j+1
                    list_pr_1.append(m/j)
                    if (n<2):
                        rank_cv.append(1/j)
                        n=n+1
            else:
                j=j+1
        if (m!=0):
            ave_prec2.append(sum(list_pr_1)/m)
        else:
            ave_prec2.append(0)
        rank2.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        for i in range(len(paper_4)):
            if paper_4[i] in test:
                m=m+1
                list_pr_1.append(m/(i+1))
                if (n<2):
                    rank_cv.append(1/(i+1))
                    n=n+1
        if (m!=0):
            ave_prec1.append(sum(list_pr_1)/m)
        else:
            ave_prec1.append(0)
        rank1.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        j=0
        for i in range(len(paper_4)):
            if paper_4[i] in list1:
                if paper_4[i] in test:
                    m=m+1
                    j=j+1
                    list_pr_1.append(m/j)
                    if (n<2):
                        rank_cv.append(1/j)
                        n=n+1
            else:
                j=j+1
        if (m!=0):
            ave_prec2.append(sum(list_pr_1)/m)
        else:
            ave_prec2.append(0)
        rank2.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        for i in range(len(paper_5)):
            if paper_5[i] in test:
                m=m+1
                list_pr_1.append(m/(i+1))
                if (n<2):
                    rank_cv.append(1/(i+1))
                    n=n+1
        if (m!=0):
            ave_prec1.append(sum(list_pr_1)/m)
        else:
            ave_prec1.append(0)
        rank1.append(sum(rank_cv))
        list_pr_1=[]
        rank_cv=[]
        n=1
        m=0
        j=0
        for i in range(len(paper_5)):
            if paper_5[i] in list1:
                if paper_5[i] in test:
                    m=m+1
                    j=j+1
                    list_pr_1.append(m/j)
                    if (n<2):
                        rank_cv.append(1/j)
                        n=n+1
            else:
                j=j+1
        if (m!=0):
            ave_prec2.append(sum(list_pr_1)/m)
        else:
            ave_prec2.append(0)
        rank2.append(sum(rank_cv))
        prec.append(exact_hit2)
        recall.append(exact_hit1)
    #     dens.append(nx.density(Pop_G))
    #     count=0
    #     for i in dict(Pop_G.degree()).items():
    #         count = count+i[1]
    #     deg.append(count/(Pop_G.order()))
    # #     ccoef.append(nx.clustering(Pop_G,Node))
    # #     ccoef1.append(nx.average_clustering(Pop_G))
    #     gccoef.append(nx.transitivity(Pop_G))
    # #     print(nx.overall_reciprocity(G_ego))
    # #     grc.append(nx.global_reaching_centrality(G_ego))
    # #     dens1.append(nx.density(Pop_G))
    #     sum1=0
    #     n_path=0
    #     for j in nx.shortest_path_length(Pop_G):
    #         for a,b in j[1].items():
    #             sum1=sum1+b
    #             if (b!=0):
    #                 n_path=n_path+1
    #     apl.append(sum1/n_path)
    prec1.append(prec)
    recall1.append(recall)
    ave_prec3.append(ave_prec1)
    rank3.append(rank1)
    ave_prec4.append(ave_prec2)
    rank4.append(rank2)
    time_exec1.append(time_exec)
    # time_exec.append((time.time()-start_time))
    # paper_1=sentences[0:10]
    # paper_2=sentences[0:15]
    # paper_3=sentences[0:20]
    # paper_4=sentences[0:25]
    # paper_5=sentences[0:30]
    # paper_6=sentences[0:100]
    # pap_ids.append(paper_6)

In [ ]:
cluster['Popularity'] = score

### Evaluation Scores

In [ ]:
node_val=[]
P_10=[]
P_15=[]
P_20=[]
P_25=[]
P_30=[]
R_10=[]
R_15=[]
R_20=[]
R_25=[]
R_30=[]
AP_10=[]
AP_15=[]
AP_20=[]
AP_25=[]
AP_30=[]
AP_10_1=[]
AP_15_1=[]
AP_20_1=[]
AP_25_1=[]
AP_30_1=[]
RA_10=[]
RA_15=[]
RA_20=[]
RA_25=[]
RA_30=[]
RA_10_1=[]
RA_15_1=[]
RA_20_1=[]
RA_25_1=[]
RA_30_1=[]
time_taken=[]
# for i in range(845):
for i in range(2000):
    node_val.append(node_5[i])
    P_10.append((prec1[i][0][0]+prec1[i][1][0]+prec1[i][2][0]))
    P_15.append((prec1[i][0][1]+prec1[i][1][1]+prec1[i][2][1]))
    P_20.append((prec1[i][0][2]+prec1[i][1][2]+prec1[i][2][2]))
    P_25.append((prec1[i][0][3]+prec1[i][1][3]+prec1[i][2][3]))
    P_30.append((prec1[i][0][4]+prec1[i][1][4]+prec1[i][2][4]))
    R_10.append((recall1[i][0][0]+recall1[i][1][0]+recall1[i][2][0])/3)
    R_15.append((recall1[i][0][1]+recall1[i][1][1]+recall1[i][2][1])/3)
    R_20.append((recall1[i][0][2]+recall1[i][1][2]+recall1[i][2][2])/3)
    R_25.append((recall1[i][0][3]+recall1[i][1][3]+recall1[i][2][3])/3)
    R_30.append((recall1[i][0][4]+recall1[i][1][4]+recall1[i][2][4])/3)
    AP_10.append((ave_prec3[i][0]+ave_prec3[i][5]+ave_prec3[i][10])/3)
    AP_15.append((ave_prec3[i][1]+ave_prec3[i][6]+ave_prec3[i][11])/3)
    AP_20.append((ave_prec3[i][2]+ave_prec3[i][7]+ave_prec3[i][12])/3)
    AP_25.append((ave_prec3[i][3]+ave_prec3[i][8]+ave_prec3[i][13])/3)
    AP_30.append((ave_prec3[i][4]+ave_prec3[i][9]+ave_prec3[i][14])/3)
    AP_10_1.append((ave_prec4[i][0]+ave_prec4[i][5]+ave_prec4[i][10])/3)
    AP_15_1.append((ave_prec4[i][1]+ave_prec4[i][6]+ave_prec4[i][11])/3)
    AP_20_1.append((ave_prec4[i][2]+ave_prec4[i][7]+ave_prec4[i][12])/3)
    AP_25_1.append((ave_prec4[i][3]+ave_prec4[i][8]+ave_prec4[i][13])/3)
    AP_30_1.append((ave_prec4[i][4]+ave_prec4[i][9]+ave_prec4[i][14])/3)
    RA_10.append((rank3[i][0]+rank3[i][5]+rank3[i][10])/3)
    RA_15.append((rank3[i][1]+rank3[i][6]+rank3[i][11])/3)
    RA_20.append((rank3[i][2]+rank3[i][7]+rank3[i][12])/3)
    RA_25.append((rank3[i][3]+rank3[i][8]+rank3[i][13])/3)
    RA_30.append((rank3[i][4]+rank3[i][9]+rank3[i][14])/3)
    RA_10_1.append((rank4[i][0]+rank4[i][5]+rank4[i][10])/3)
    RA_15_1.append((rank4[i][1]+rank4[i][6]+rank4[i][11])/3)
    RA_20_1.append((rank4[i][2]+rank4[i][7]+rank4[i][12])/3)
    RA_25_1.append((rank4[i][3]+rank4[i][8]+rank4[i][13])/3)
    RA_30_1.append((rank4[i][4]+rank4[i][9]+rank4[i][14])/3)
    time_taken.append(statistics.mean(time_exec1[i]))  

In [ ]:
reccv_eval2 = pd.DataFrame({
    'Paper_Id':node_val,
    'Rec@10':R_10,
    'Prec@10':P_10,
    'AP@10':AP_10,
    'RR@10':RA_10,
    'AP1@10':AP_10_1,
    'RR1@10':RA_10_1,
    'Rec@15':R_15,
    'Prec@15':P_15,
    'AP@15':AP_15,
    'RR@15':RA_15,
    'AP1@15':AP_15_1,
    'RR1@15':RA_15_1,
    'Rec@20':R_20,
    'Prec@20':P_20,
    'AP@20':AP_20,
    'RR@20':RA_20,
    'AP1@20':AP_20_1,
    'RR1@20':RA_20_1,
    'Rec@25':R_25,
    'Prec@25':P_25,
    'AP@25':AP_25,
    'RR@25':RA_25,
    'AP1@25':AP_25_1,
    'RR1@25':RA_25_1,
    'Rec@30':R_30,
    'Prec@30':P_30,
    'AP@30':AP_30,
    'RR@30':RA_30,
    'AP1@30':AP_30_1,
    'RR1@30':RA_30_1,
    'Time':time_taken
})
